In [1]:
import pandas as pd
import random
import names
import datetime as dt
import sqlite3

In [2]:
# Converts a 4 digit number into a 5 digit checksum
def checkSum(num):
    numString = str(num)
    checkNum = str(3 * (int(numString[0]) + int(numString[2])) + 7 * (int(numString[1]) + int(numString[3])))[-1]
    numString += checkNum
    return int(numString)

def randomDateString():
    minDate = dt.date(2015,1,1)
    maxDate = dt.date(2022,10,1)
    dateRange = (maxDate-minDate).days
    randDate = minDate + dt.timedelta(days=random.randrange(dateRange))
    return randDate.strftime("%Y-%m-%d")

def randomDayAdder(maxYear, yearIn):
    date = dt.datetime.strptime(yearIn,"%Y-%m-%d")
    if date.year < maxYear:
        return (date + dt.timedelta(days=random.randint(4,20))).strftime("%Y-%m-%d")
    else:
        return None
    

# List of genres
genreList = ["Horror","Comedy","Thriller","Fantasy","Non-Fiction","Cooking","History"]

# List of residences
residences = ["Como","Subiaco","Fremantle","Rockingham","Midland","Maylands","Joondalup","Kingsley"]

In [3]:
# Create sample data for BookEdition
isbn = [checkSum(n) for n in random.sample(range(1000,10000), 100)]
author = [names.get_full_name() for n in range(100)]
publicationDate = [random.randint(1900,2022) for n in range(100)]
genre = [genreList[random.randint(0,len(genreList)-1)] for n in range(100)]

BookEdition = pd.DataFrame(list(zip(isbn,author,publicationDate,genre)),
                          columns=["ISBN","author","publicationDate","genre"])

In [4]:
BookEdition.head()

,ISBN,author,publicationDate,genre
0,67409,Lucinda Coleman,1991,Cooking
1,51686,Amber Holland,1912,Cooking
2,47991,Frank Ferguson,1973,Fantasy
3,56519,Rosita Martin,1925,Horror
4,48644,Jerry Baker,1996,Thriller


In [5]:
# Create sample data for BookCopy
isbn_copy = [isbn[random.randint(0,len(isbn)-1)] for n in range(200)]
copyNumber = [isbn_copy[0:i+1].count(item) for i, item in enumerate(isbn_copy)]
daysLoaned = [0] * len(isbn_copy)

BookCopy = pd.DataFrame(list(zip(isbn_copy,copyNumber,daysLoaned)),
                       columns=["ISBN","copyNumber","daysLoaned"])

In [6]:
BookCopy.head()

,ISBN,copyNumber,daysLoaned
0,68932,1,0
1,14678,1,0
2,54883,1,0
3,31051,1,0
4,16775,1,0


In [7]:
# Create sample data for Client
clientId = [i+1 for i in range(60)]
name = [names.get_full_name() for n in range(60)]
residence = [residences[random.randint(0,len(residences)-1)] for n in range(60)]

Client = pd.DataFrame(list(zip(clientId,name,residence)),
                     columns=["clientId","name","residence"])

In [8]:
Client.head()

,clientId,name,residence
0,1,John Harbeson,Joondalup
1,2,Charlotte Martin,Subiaco
2,3,Shane Froedge,Subiaco
3,4,Gary Broussard,Rockingham
4,5,James Morris,Kingsley


In [9]:
# List of tuples ensures ISBN and copyNumber are consistent and not mismatched during random selection
isbn_copyNumber = [list(zip(isbn_copy,copyNumber))[random.randint(0,len(isbn_copy)-1)] for n in range(500)]

clientId_loan = [clientId[random.randint(0,59)] for n in range(500)]
isbn_loan = [item[0] for item in isbn_copyNumber]
copyNumber_loan = [item[1] for item in isbn_copyNumber]
dateOut = [randomDateString() for n in range(500)]
dateBack = [None for n in range(500)]

loan = pd.DataFrame(list(zip(clientId_loan,isbn_loan,copyNumber_loan,dateOut,dateBack)),
                   columns=["clientId","ISBN","copyNumber","dateOut","dateBack"])

In [10]:
loan.head()

,clientId,ISBN,copyNumber,dateOut,dateBack
0,40,22251,2,2021-06-17,None
1,57,14678,1,2016-07-09,None
2,23,71149,1,2015-02-01,None
3,51,37400,1,2016-11-13,None
4,20,26383,1,2019-03-27,None


In [11]:
# Add sample data to sqlite tables
conn = sqlite3.connect("test.db")
cursor = conn.cursor()

In [12]:
BookEdition.to_sql(name="BookEdition", con=conn, if_exists="append", index=False)

100

In [13]:
BookCopy.to_sql(name="BookCopy", con=conn, if_exists="append", index=False)

200

In [14]:
Client.to_sql(name="Client", con=conn, if_exists="append", index=False)

60

In [15]:
loan.to_sql(name="loan", con=conn, if_exists="append", index=False)

500

In [16]:
# Commit to adding sample data
conn.commit()

In [17]:
# Update dateBack column in loan table to check trigger
newDateBack = [randomDayAdder(2022, date) for date in dateOut]

# Append rows to sqlite tables
for items in zip(newDateBack, isbn_loan, copyNumber_loan, clientId_loan):
    if items[0]:
        insert = f"UPDATE loan SET dateBack = '{items[0]}' WHERE ISBN = '{items[1]}' AND copyNumber = '{items[2]}' AND clientId = {items[3]} AND dateBack IS NULL AND dateOut < '{items[0]}';"
        cursor.execute(insert)
        conn.commit()